In [1]:
class Account:
    month = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    season = ["ns", "ns", "ns", "ns", "s", "s", "s", "s", "s", "s", "ns", "ns"]
    user_type = {
            "la" : "la",
            "lv" : "lv",
            "hv" : "hv"
        }

    def __init__(self, id, name):
        self.id = id
        self.name = name
        self.contract = {
            "s" : {"r" : 0, "hp" :0, "ns": 0, "shp" : 0,"op" : 0},
            "ns" : {"r" : 0, "hp" : 0, "ns" : 0,"shp" : 0,"op" : 0}
        }
        
        self.demand = {}

        self.unitcprice = {
            "s" : {"r" : 0, "hp" :0, "ns": 0, "shp" : 0,"op" : 0},
            "ns" : {"r" : 0, "hp" : 0, "ns" : 0,"shp" : 0,"op" : 0}
        }
        
        self.baseprice = {}

    def setContract(self, season="s", c_r=0, c_hp=0, c_ns = 0, c_shp = 0, c_op = 0): 
        if c_r <= 1:
            print('must be larger than 1')
        self.contract[season]["r"] = c_r
        self.contract[season]["hp"] = c_hp
        self.contract[season]["ns"] = c_ns
        self.contract[season]["shp"] = c_shp
        self.contract[season]["op"] = c_op

        
    #Method for input the demand
    #demand_r: regular demand
    #demand_ns = non-summer demand
    #demand_sat = Saturday demand
    #demant_op = off-peak demand
    def setDemand(self, month_index=0, d_r=0, d_hp=0, d_ns = 0, d_shp = 0, d_op = 0): 
        if d_r <= 0:
            print('must be positive')
        self.demand[month_index] = {
            "month" : self.month[month_index],
            "season" : self.season[month_index],
            "r" : d_r,
            "hp" : d_hp,
            "ns" : d_ns,
            "shp" : d_shp,
            "op" : d_op
            }
        
    def setUnitcontractprice(self, season = "s", ucpr_r = 0, ucpr_hp = 0, ucpr_ns = 0, ucpr_sat = 0, ucpr_op = 0): 
        self.unitcprice[season]["r"] = ucpr_r
        self.unitcprice[season]["hp"] = ucpr_hp
        self.unitcprice[season]["ns"] = ucpr_ns
        self.unitcprice[season]["shp"] = ucpr_sat
        self.unitcprice[season]["op"] = ucpr_op

    def getContract(self, month_index):
        s = self.season[month_index]
        contract_month = {
            "r" : self.contract[s]["r"],
            "hp" : self.contract[s]["hp"],
            "ns" : self.contract[s]["ns"],
            "shp" : self.contract[s]["shp"],
            "op" : self.contract[s]["op"]
        }
        return contract_month

    def getDemand(self, month_index):
        demand_month = {
            "r" : self.demand[month_index]["r"],
            "hp" : self.demand[month_index]["hp"],
            "ns" : self.demand[month_index]["ns"],
            "shp" : self.demand[month_index]["shp"],
            "op" : self.demand[month_index]["op"]
        }
        return demand_month
    
    def getUnitcontractprice(self, month_index):
        s = self.season[month_index]
        unitcprice_month = {
            "r" : self.unitcprice[s]["r"],
            "hp" : self.unitcprice[s]["hp"],
            "ns" : self.unitcprice[s]["ns"],
            "shp" : self.unitcprice[s]["shp"],
            "op" : self.unitcprice[s]["op"]
        }
        return unitcprice_month
        
    def cacuExceedcontract(self, contract, demand):
        cacuEx = lambda x, y : x-y if x-y > 0 else 0 
        exceed = {
            "r" : cacuEx(demand["r"],contract["r"]),
            "hp" :cacuEx(demand["hp"],contract["r"]+contract["hp"]),
            "ns" : cacuEx(demand["ns"],contract["r"]+contract["ns"]),
            "shp" : cacuEx(demand["shp"],contract["r"]+contract["ns"]+contract["hp"]+contract["shp"]),
            "op" : cacuEx(demand["op"],contract["r"]+contract["ns"]+contract["hp"]+contract["shp"]+contract["op"])
            }
        return exceed
    
    
    def getBaseprice(self, contract, unitcprice):
        common = contract["shp"]+contract["op"]-(contract["r"]+contract["ns"])*0.5
        return contract["r"]*unitcprice["r"]+contract["ns"]*unitcprice["ns"]+contract["hp"]*unitcprice["hp"]+contract["shp"]*common
    

    def getPenaltyprice(self, contract, exceed, unitcprice):
        cacuStage_1 = lambda x, y: x - y*0.1

        hp = max(exceed["hp"]-exceed["r"], 0)
        shp = max(exceed["shp"]-max(exceed["r"], exceed["hp"]), 0)
        op = max(exceed["op"]-max(exceed["r"],exceed["hp"], exceed["shp"]), 0)

        penalty = {
            "r" : {
                "stage_1" : cacuStage_1(exceed["r"], contract["r"]),
                "stage _2" : exceed["r"] - penaltyprice["r"]["stage_1"]
            },
            "hp" : {
                "stage_1" : cacuStage_1(hp, contract["hp"]),
                "stage _2" : hp - penaltyprice["hp"]["stage_1"]
            },
            "ns" : {
                "stage_1" : cacuStage_1(exceed["ns"], contract["ns"]),
                "stage _2" : exceed["ns"] - penalty["ns"]["stage_1"]
            },
            "shp" : {
                "stage_1" : cacuStage_1(shp, contract["shp"]),
                "stage _2" : shp - penalty["shp"]["stage_1"]
            },
            "op" : {
                "stage_1" : cacuStage_1(op, contract["op"]),
                "stage _2" : op - penalty["op"]["stage_1"]
            }
        }


        
        first = exceed - round(contract["r"] * 0.1, 0)
        second = exceed - first
  
        penaltyprice = {
            "first" : first * unitcprice["r"] * 2,
            "second" : second * unitcprice * 3
        } 
        return penaltyprice

    
    def cacuTotal(self):
        for month_index in range(12):
            contract = self.getContract(month_index)
            demand = self.getDemand(month_index)
            exceed = self.cacuExceedcontract(contract, demand)
            unitcprice = self.getUnitcontractprice(month_index)

            self.baseprice[month_index] = self.getBaseprice(contract,unitcprice) + self.getPenaltyprice(contract, exceed, unitcprice)

In [2]:
user1 = Account(1000,'C1')


In [3]:
user1.setUnitcontractprice("s", 223.6,0,166.9,44.7,44.7)
user1.setUnitcontractprice("ns", 166.9,0,166.9,33.3,33.3)
user1.unitcprice

{'s': {'r': 223.6, 'hp': 0, 'ns': 166.9, 'shp': 44.7, 'op': 44.7},
 'ns': {'r': 166.9, 'hp': 0, 'ns': 166.9, 'shp': 33.3, 'op': 33.3}}

In [4]:
user1.setContract("s", 300)
user1.setContract("ns", 300)
user1.contract

{'s': {'r': 300, 'hp': 0, 'ns': 0, 'shp': 0, 'op': 0},
 'ns': {'r': 300, 'hp': 0, 'ns': 0, 'shp': 0, 'op': 0}}

In [5]:
demand = [[68,0,0,62,36],
          [66,0,0,42,37],
          [177,0,0,154,36],
          [278,0,0,179,150],
          [276,0,0,164,39],
          [309,0,0,201,40],
          [357,0,0,215,48],
          [329,0,0,205,174],
          [342,0,0,312,54],
          [326,0,0,234,47],
          [240,0,0,173,92],
          [157,0,0,153,36]]

for month_index in range(12):
   user1.setDemand(month_index, demand[month_index][0],demand[month_index][1], demand[month_index][2],demand[month_index][3],demand[month_index][4])
   print(user1.demand[month_index])

{'month': 'Jan', 'season': 'ns', 'r': 68, 'hp': 0, 'ns': 0, 'shp': 62, 'op': 36}
{'month': 'Feb', 'season': 'ns', 'r': 66, 'hp': 0, 'ns': 0, 'shp': 42, 'op': 37}
{'month': 'Mar', 'season': 'ns', 'r': 177, 'hp': 0, 'ns': 0, 'shp': 154, 'op': 36}
{'month': 'Apr', 'season': 'ns', 'r': 278, 'hp': 0, 'ns': 0, 'shp': 179, 'op': 150}
{'month': 'May', 'season': 's', 'r': 276, 'hp': 0, 'ns': 0, 'shp': 164, 'op': 39}
{'month': 'Jun', 'season': 's', 'r': 309, 'hp': 0, 'ns': 0, 'shp': 201, 'op': 40}
{'month': 'Jul', 'season': 's', 'r': 357, 'hp': 0, 'ns': 0, 'shp': 215, 'op': 48}
{'month': 'Aug', 'season': 's', 'r': 329, 'hp': 0, 'ns': 0, 'shp': 205, 'op': 174}
{'month': 'Sep', 'season': 's', 'r': 342, 'hp': 0, 'ns': 0, 'shp': 312, 'op': 54}
{'month': 'Oct', 'season': 's', 'r': 326, 'hp': 0, 'ns': 0, 'shp': 234, 'op': 47}
{'month': 'Nov', 'season': 'ns', 'r': 240, 'hp': 0, 'ns': 0, 'shp': 173, 'op': 92}
{'month': 'Dec', 'season': 'ns', 'r': 157, 'hp': 0, 'ns': 0, 'shp': 153, 'op': 36}


In [6]:
user1.cacuTotal()
user1.baseprice

TypeError: 'int' object is not iterable